<a href="https://colab.research.google.com/github/laskari/END-Program/blob/main/Assignment_Sentiment_Analysis_using_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import csv
sentences={}
with open("/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/datasetSentences.txt","r") as f:
  rd=csv.reader(f, delimiter='\t')
  count=0
  for line in rd:
    if count==0:
      count=1
      continue
    line[1]=line[1].replace('-LRB-','(')
    line[1]=line[1].replace('-RRB-',')')
    line[1]=line[1].replace('Â', '')
    line[1]=line[1].replace('Ã©', 'e')
    line[1]=line[1].replace('Ã¨', 'e')
    line[1]=line[1].replace('Ã¯', 'i')
    line[1]=line[1].replace('Ã³', 'o')
    line[1]=line[1].replace('Ã´', 'o')
    line[1]=line[1].replace('Ã¶', 'o')
    line[1]=line[1].replace('Ã±', 'n')
    line[1]=line[1].replace('Ã¡', 'a')
    line[1]=line[1].replace('Ã¢', 'a')
    line[1]=line[1].replace('Ã£', 'a')
    line[1]=line[1].replace('\xc3\x83\xc2\xa0', 'a')
    line[1]=line[1].replace('Ã¼', 'u')
    line[1]=line[1].replace('Ã»', 'u')
    line[1]=line[1].replace('Ã§', 'c')
    line[1]=line[1].replace('Ã¦', 'ae')
    line[1]=line[1].replace('Ã­', 'i')
    line[1]=line[1].replace('\xa0', ' ')
    line[1]=line[1].replace('\xc2', '')
    
    sentences[line[0]]=line[1]

In [18]:
train={}
sents=[]

with open("/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/datasetSplit.txt","r") as f:
	rd=csv.reader(f, delimiter=',')
	count=0
	for line in rd:
		if count==0:
			count=1
			continue
		if line[1]=='1':
			train[sentences[line[0]]]=0
			sents.append(sentences[line[0]])

train_sent = train.copy()
string=" ".join(sents)

In [ ]:
!pip install googletrans==3.1.0a0

In [39]:
import random
import googletrans
from googletrans import Translator


def back_translate(sentence):
  translator = Translator()
  available_langs = list(googletrans.LANGUAGES.keys())
  trans_lang = random.choice(available_langs) 
  translations = translator.translate(sents, dest=trans_lang) 
  t_text = [t.text for t in translations]
  translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
  en_text = [t.text for t in translations_en_random]
  return en_text

In [40]:
result = back_translate('how are you')


KeyboardInterrupt: ignored

In [20]:
with open("/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/dictionary.txt","r") as f:
	rd=csv.reader(f,delimiter='|')
	for line in rd:
		line[0]=line[0].replace('é','e')
		line[0]=line[0].replace('è','e')
		line[0]=line[0].replace('ï','i')
		line[0]=line[0].replace('í','i')
		line[0]=line[0].replace('ó','o')
		line[0]=line[0].replace('ô','o')
		line[0]=line[0].replace('ö','o')
		line[0]=line[0].replace('á','a')
		line[0]=line[0].replace('â','a')
		line[0]=line[0].replace('ã','a')
		line[0]=line[0].replace('à','a')
		line[0]=line[0].replace('ü','u')
		line[0]=line[0].replace('û','u')
		line[0]=line[0].replace('ñ','n')
		line[0]=line[0].replace('ç','c')
		line[0]=line[0].replace('æ','ae')
		line[0]=line[0].replace('\xa0', ' ')
		line[0]=line[0].replace('\xc2', '')
		if line[0] in string:
			train[line[0]]=line[1]
		if line[0] in train_sent:
			train_sent[line[0]]=line[1]

In [21]:
labels={}
with open("/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/sentiment_labels.txt","r") as f:
	rd=csv.reader(f,delimiter='|')
	count=0
	for line in rd:
		if count==0:
			count=1
			continue
		labels[line[0]]=float(line[1])

for key in train:
  train[key]=labels[train[key]]
for key in train_sent:
	train_sent[key]=labels[train_sent[key]]

In [22]:
with open("/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/sst_train_phrases.csv","w") as f:
	wr=csv.writer(f,delimiter=',')
	for key in train:
		wr.writerow([train[key],key])
with open("/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/sst_train_sentences.csv","w") as f:
	wr=csv.writer(f,delimiter=',')
	for key in train_sent:
		wr.writerow([train_sent[key],key])

In [23]:
with open("/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/sst5_train_sentences.csv","w") as f:
	wr=csv.writer(f,delimiter=',')
	for key in train_sent:
		x=train_sent[key]
		if x<=0.2:
			x='very neg'
		elif x<=0.4:
			x='neg'
		elif x<=0.6:
			x='neu'
		elif x<=0.8:
			x='pos'
		elif x<=1:
			x='very pos'
		wr.writerow([key,x])

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/END School of AI/Datasets/stanfordSentimentTreebank/sst5_train_sentences.csv', names = ['Sentence','label'])
df.head()

,Sentence,label
0,The Rock is destined to be the 21st Century 's...,pos
1,The gorgeously elaborate continuation of `` Th...,very pos
2,Singer\/composer Bryan Adams contributes a sle...,pos
3,You 'd think by now America would have had eno...,neu
4,Yet the act is still charming here .,pos


In [2]:
print(df.shape)
df.label.values

(8534, 2)


array(['pos', 'very pos', 'pos', ..., 'pos', 'very neg', 'neg'],
      dtype=object)

In [3]:
df.label.value_counts()

pos         2319
neg         2215
neu         1623
very pos    1287
very neg    1090
Name: label, dtype: int64

In [4]:
def random_swap(sentence, n=5): 
    sentence = sentence.split()
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [5]:
def random_deletion(words, p=0.5): 
    words = words.split()
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,len(words)) > p, words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [ ]:
import random
df['Sentence'] = df.Sentence.apply(lambda x: random_swap(x))
df['Sentence'] =  ' '.join([str(elem) for elem in df['Sentence']])
df['Sentence'] = df.Sentence.apply(lambda x: random_deletion(x))
df['Sentence'] =  ' '.join([str(elem) for elem in df['Sentence']])

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [ ]:
fields = [('Sentence', Sentence),('label',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [ ]:
example = [data.Example.fromlist([df.Sentence[i],df.label[i]], fields) for i in range(df.shape[0])] 

In [ ]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [ ]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
(len(train), len(valid))

(7254, 1280)

An example from the dataset:

In [ ]:
vars(train.examples[10])

{'Sentence': ['Even',
  'by',
  'the',
  'intentionally',
  'low',
  'standards',
  'of',
  'frat',
  '-',
  'boy',
  'humor',
  ',',
  'Sorority',
  'Boys',
  'is',
  'a',
  'bowser',
  '.'],
 'label': 'very neg'}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [ ]:
Sentence.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [ ]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  15810
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 6824), (',', 6055), ('the', 5199), ('and', 3808), ('of', 3794), ('a', 3729), ('to', 2549), ('-', 2360), ("'s", 2180), ('is', 2156)]
Labels :  defaultdict(<function _default_unk_index at 0x7fe2a499e048>, {'pos': 0, 'neg': 1, 'neu': 2, 'very pos': 3, 'very neg': 4})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.Sentence),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(15810, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 4,985,105 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.Sentence   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.Sentence
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [ ]:
N_EPOCHS = 100
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.582 | Train Acc: 27.80%
	 Val. Loss: 1.575 |  Val. Acc: 28.20% 

	Train Loss: 1.568 | Train Acc: 29.97%
	 Val. Loss: 1.571 |  Val. Acc: 29.53% 

	Train Loss: 1.545 | Train Acc: 35.15%
	 Val. Loss: 1.549 |  Val. Acc: 33.91% 

	Train Loss: 1.500 | Train Acc: 40.02%
	 Val. Loss: 1.539 |  Val. Acc: 34.69% 

	Train Loss: 1.452 | Train Acc: 45.21%
	 Val. Loss: 1.530 |  Val. Acc: 35.47% 

	Train Loss: 1.410 | Train Acc: 49.90%
	 Val. Loss: 1.522 |  Val. Acc: 37.19% 

	Train Loss: 1.370 | Train Acc: 55.04%
	 Val. Loss: 1.525 |  Val. Acc: 35.55% 

	Train Loss: 1.334 | Train Acc: 58.80%
	 Val. Loss: 1.531 |  Val. Acc: 36.17% 

	Train Loss: 1.302 | Train Acc: 61.80%
	 Val. Loss: 1.530 |  Val. Acc: 35.55% 

	Train Loss: 1.272 | Train Acc: 64.43%
	 Val. Loss: 1.542 |  Val. Acc: 33.05% 

	Train Loss: 1.243 | Train Acc: 67.99%
	 Val. Loss: 1.530 |  Val. Acc: 35.70% 

	Train Loss: 1.214 | Train Acc: 71.31%
	 Val. Loss: 1.542 |  Val. Acc: 33.98% 

	Train Loss: 1.187 | Train Acc: 73.76%
	

## Model Testing

In [ ]:
#load weights and tokenizer

path='/content/saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('/content/tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    #{'pos': 0, 'neg': 1, 'neu': 2, 'very pos': 3, 'very neg': 4}
    categories = {0: "Positive", 1:"Negative", 2:"Neutral",3: "Very Positive", 4:"Very Negative"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Positive'

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [ ]:
def random_deletion(words, p=0.5): 
    words = words.split()
    print(len(words))
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,len(words)) > p, words)) 
    print(remaining)
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [ ]:
def random_swap(sentence, n=5): 
    sentence = sentence.split()
    length = range(len(sentence)) 
    print(length)
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        print(idx1, idx2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [ ]:
new = random_swap('Welcome to the Deep NLP Program using Python and Pytorch with the School of AI', 5)
print(' '.join(new))


new_del = random_deletion('Welcome to the Deep NLP Program using Python and Pytorch with the School of AI', 5)
print(' '.join(new_del))

range(0, 15)
0 4
11 12
2 7
5 10
11 12
NLP to Python Deep Welcome with using the and Pytorch Program the School of AI
15
['to', 'the', 'Deep', 'NLP', 'Program', 'using', 'Python', 'and', 'Pytorch']
to the Deep NLP Program using Python and Pytorch


For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 